Train the BPE tokenizer on a small sample of the datasets, add special tokens.

IMPROVEMENTS:
- prevent special token injection

Train the BPE tokenizer on a small sample of the datasets, add special tokens.

IMPROVEMENTS:
- prevent special token injection
- retrain on the new wikipedia datasetm

In [1]:
from tokenizer_datasets import *

lite_dataset = False
dataset_iter, dataset_length = tokenizer_lite_dataset() if lite_dataset else tokenizer_real_dataset()

print(dataset_length)
example_message = next(dataset_iter())
print(example_message)

Repo card metadata block was not found. Setting CardData to empty.


3697838982
Aster is a chatbot who answers questions with rhymes.


In [2]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

tokenizer = Tokenizer(models.BPE(unk_token="<|unk|>"))

In [3]:
from tokenizers.normalizers import NFD, StripAccents, Sequence

tokenizer.normalizer = Sequence([
    NFD(),
    StripAccents(),
])
tokenizer.normalizer.normalize_str("Héllò hôw are ü?")

'Hello how are u?'

In [4]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer.pre_tokenize_str(example_message)

[('Aster', (0, 5)),
 ('Ġis', (5, 8)),
 ('Ġa', (8, 10)),
 ('Ġchatbot', (10, 18)),
 ('Ġwho', (18, 22)),
 ('Ġanswers', (22, 30)),
 ('Ġquestions', (30, 40)),
 ('Ġwith', (40, 45)),
 ('Ġrhymes', (45, 52)),
 ('.', (52, 53))]

In [5]:
from special_tokens import special_tokens

special_token_list = list(special_tokens.values())
tokenizer.add_tokens(special_token_list)
tokenizer.get_vocab()

{'<|assistant|>': 5,
 '<|eos|>': 3,
 '<|pad|>': 7,
 '<|user|>': 4,
 '<|system|>': 6,
 '<|bos|>': 2,
 '<|unk|>': 8,
 '<|endoftext|>': 0,
 '<|endofturn|>': 1}

In [7]:
from settings import ModelSettings

trainer = trainers.BpeTrainer(
    vocab_size=ModelSettings.vocabulary_size,
    special_tokens=special_token_list,
    show_progress=True,
    min_frequency=2,
)

In [8]:
tokenizer.train_from_iterator(dataset_iter(), trainer, length=dataset_length)
file_name = "./tokenizer.json"

no robots completed
wiki completed
tiny stories completed
tiny textbooks completed





In [9]:
from tokenizers import decoders, processors

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [10]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="<|bos|> $A <|eos|>",  # adds BOS before & EOS after each sequence
    pair="<|bos|> $A <|eos|> <|bos|> $B <|eos|>",  # for pairs (less common)
    special_tokens=[
        ("<|bos|>", tokenizer.token_to_id("<|bos|>")),
        ("<|eos|>", tokenizer.token_to_id("<|eos|>")),
    ],
)

In [11]:
tokenizer.save(file_name)

In [12]:
print("<|user|>" in tokenizer.get_vocab())
print("<|" in tokenizer.get_vocab())

True
False


In [13]:
tokenizer = Tokenizer.from_file(file_name)

encoding = tokenizer.encode(example_message)
print(encoding.tokens)
decoding = tokenizer.decode(encoding.ids)
print(decoding)

['<|bos|>', 'A', 'ster', 'Ġis', 'Ġa', 'Ġchat', 'bot', 'Ġwho', 'Ġanswers', 'Ġquestions', 'Ġwith', 'Ġrh', 'ym', 'es', '.', '<|eos|>']
Aster is a chatbot who answers questions with rhymes.
